In [0]:
# Set up parameters with widgets
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema", "")
dbutils.widgets.text("volume", "")

# Read the widget values
catalog = 'mido_edw_dev' #dbutils.widgets.get("catalog")
schema = 'sentiment_analysis' #dbutils.widgets.get("schema")
volume = dbutils.widgets.get("volume")

In [0]:
%pip install datasets


In [0]:
import datasets
datasets.logging.set_verbosity_error()



In [0]:
from datasets import load_dataset, concatenate_datasets

from pyspark.sql.utils import AnalysisException

table_name = f"{catalog}.{schema}.amazon_reviews"

if not spark.catalog.tableExists(table_name):
    datasetnew = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Health_and_Personal_Care", trust_remote_code=True)
    merged_dataset = concatenate_datasets([ds for ds in datasetnew.values()])
    df = merged_dataset.to_pandas()
    spark_df = spark.createDataFrame(df)
    spark_df.write.mode("overwrite").saveAsTable(table_name)
    print (f"Table {table_name} created successfully")
else:
    print (f"Table {table_name} already exists")
    


In [0]:
from datasets import load_dataset, concatenate_datasets
from pyspark.sql.functions import col, when, array
from pyspark.sql.types import ArrayType, StringType

table_name = f"{catalog}.{schema}.amazon_items"

if not spark.catalog.tableExists(table_name):
    datasetnew = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Health_and_Personal_Care", trust_remote_code=True)
    merged_dataset = concatenate_datasets([ds for ds in datasetnew.values()])
    df = merged_dataset.to_pandas()
    spark_df = spark.createDataFrame(df)
    
    # Handle NullType in categories column
    spark_df = spark_df.withColumn(
        "categories",
        when(col("categories").isNull(), array().cast(ArrayType(StringType()))).otherwise(col("categories"))
    )

    # Select only the required columns
    selected_columns = ["main_category", "title", "parent_asin", "rating_number", "average_rating"]
    spark_df = spark_df.select(*selected_columns)
    
    
    spark_df.write.mode("overwrite").saveAsTable(table_name)
    print(f"Table {table_name} created successfully")
else:
    print(f"Table {table_name} already exists")